In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import os

In [5]:
src_dir_training = "C:\\Users\\hsr30\\Documents\\minor-project\\datasets\\disease-grading-custom\\rendered-training-3500"

out_dir_training = "C:\\Users\\hsr30\\Documents\\minor-project\\datasets\\disease-grading-custom\\final-training"

src_dir_test= "C:\\Users\\hsr30\\Documents\\minor-project\\datasets\\disease-grading-custom\\rendered-testing-3500"

out_dir_test = "C:\\Users\\hsr30\\Documents\\minor-project\\datasets\\disease-grading-custom\\final-testing"

In [6]:
def circular_crop(path, out_dir):
    image = cv2.imread(path)
    base_name = os.path.basename(path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.medianBlur(gray, 11)
    thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

    # Morph open 
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=3)

    # Find contours and filter using contour area and aspect ratio
    cnts = cv2.findContours(opening, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    for c in cnts:
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.04 * peri, True)
        area = cv2.contourArea(c)
        if len(approx) > 5 and area > 1000:
            ((x, y), r) = cv2.minEnclosingCircle(c)
            # crop ROI
            x,y,r = int(x), int(y), int(r)
            ROI = image[y-r:y+r, x-r:x+r]
            cv2.imwrite(os.path.join(out_dir, base_name), ROI)
            break



In [7]:
progress = 0
total = len(os.listdir(src_dir_training))
for image in os.listdir(src_dir_training):
    circular_crop(os.path.join(src_dir_training, image), out_dir_training)
    progress += 1
    print(f"Progress: {progress}/{total}", end="\r")

In [8]:
progress = 0
total = len(os.listdir(src_dir_test))
for image in os.listdir(src_dir_test):
    circular_crop(os.path.join(src_dir_test, image), out_dir_test)
    progress += 1
    print(f"Progress: {progress}/{total}", end="\r")